In [1]:
import pandas as pd
import numpy as np
import os
from utilities.utils import ErrorFunctions, SSPModelForCalibration
from utilities.diff_reports import DiffReportUtils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Define paths
curr_dir = os.getcwd()
misc_dir = os.path.join(curr_dir, 'misc')
sectoral_report_dir = os.path.join(misc_dir, 'sectoral_reports')
dummy_files_dir = os.path.join(misc_dir, 'dummy')
sectoral_report_mapping_dir = os.path.join(misc_dir, 'sectoral_report_mapping')


In [4]:
# Initialize the diff report object
dru = DiffReportUtils('HRV', os.path.join(sectoral_report_mapping_dir, 'edgar_ssp_cw.csv'), sectoral_report_dir, energy_model_flag=False)

In [5]:
# Load the edgar emissions database for the desired region
edgar_emissions = dru.edgar_emission_db_etl(os.path.join(sectoral_report_mapping_dir, 'CSC-GHG_emissions-April2024_to_calibrate.csv'))
edgar_emissions

,iso_alpha_3,edgar_class,edgar_emission,year
0,HRV,AG - Livestock:CH4,1.682916,2015
1,HRV,AG - Livestock:N2O,0.110193,2015
2,HRV,AG - Crops:CH4,0.000227,2015
3,HRV,AG - Crops:CO2,0.095857,2015
4,HRV,AG - Crops:N2O,1.038055,2015
5,HRV,EN - Building:CH4,0.416708,2015
6,HRV,EN - Building:CO2,2.726867,2015
7,HRV,EN - Building:N2O,0.104122,2015
8,HRV,EN - Electricity/Heat:CH4,0.004396,2015
9,HRV,EN - Electricity/Heat:CO2,3.297328,2015


## Test Diff Report Class Methods

In [6]:
ssp_out = pd.read_csv(os.path.join(dummy_files_dir, 'ssp_output_no_energy_dummy.csv'))

In [7]:
ssp_out.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,128992.475401,286925.570284,59534.988647,155176.845408,200241.524314,109285.291660,144565.284932,96468.731604,66425.612333,...,860714.235864,5.781814e+06,65829.816853,279108.581486,50211.974800,175242.943363,0.0,0.0,2.050396e+06,725849.288265
1,1,128305.935425,285398.458926,59218.124042,154350.943684,199175.773688,108703.639735,143795.861390,95955.293587,66072.073584,...,857478.660027,5.906717e+06,52166.876709,331312.854435,70450.376552,215192.338859,2736.0,2449.0,2.713049e+06,699533.070417
2,2,127621.990578,283877.120067,58902.457190,153528.163879,198114.051612,108124.186462,143029.346278,95443.796372,65719.871216,...,660924.245615,5.951148e+06,38967.956801,296921.771405,82224.830575,210243.344677,2736.0,2449.0,2.331104e+06,761148.281366
3,3,126940.639041,282361.549661,58587.987250,152708.503803,197056.355263,107546.930298,142265.737557,94934.238599,65369.004292,...,957822.768981,5.919376e+06,17239.772927,378730.362142,93529.211867,213539.618942,2736.0,2449.0,2.111415e+06,729155.981592
4,4,126261.879236,280851.744198,58274.713494,151891.961560,196002.682190,106971.869908,141505.033460,94426.619087,65019.472000,...,892950.504956,5.887725e+06,21001.252360,359927.486275,85408.876964,264501.812874,2736.0,2449.0,2.234268e+06,737682.556359


In [8]:
?dru.load_ssp_edgar_cw

Signature: dru.load_ssp_edgar_cw()
Docstring:
Load the SSP Edgar CW data from a CSV file.
This method reads the SSP Edgar CW data from the CSV file specified by 
the `ssp_edgar_cw_path` attribute and returns it as a pandas DataFrame.
Returns:
    pd.DataFrame: The SSP Edgar CW data loaded from the CSV file.
File:      ~/decision_sciences/ssp_data_calibration/src/utilities/diff_reports.py
Type:      method

In [10]:
ssp_edgar_df = dru.load_ssp_edgar_cw()
ssp_edgar_df.tail(20)

,subsector,gas,edgar_class,edgar_subsector,edgar_sector,vars
48,ippu,chf3,IN - Industrial Processes:HFC,IN - Industrial Processes,Industrial Processes,emission_co2e_chf3_ippu_product_use_product_us...
49,ippu,hfcs,IN - Industrial Processes:HFC,IN - Industrial Processes,Industrial Processes,emission_co2e_hfcs_ippu_product_use_product_us...
50,ippu,other_fcs,IN - Industrial Processes:HFC,IN - Industrial Processes,Industrial Processes,emission_co2e_other_fcs_ippu_product_use_produ...
51,ippu,pfcs,IN - Industrial Processes:PFC,IN - Industrial Processes,Industrial Processes,emission_co2e_pfcs_ippu_product_use_product_us...
52,ippu,sf6,IN - Industrial Processes:SF6,IN - Industrial Processes,Industrial Processes,emission_co2e_sf6_ippu_production_chemicals:em...
53,waso,co2,Waste - Solid Waste:CO2,Waste - Solid Waste,Waste,emission_co2e_co2_waso_incineration
54,waso,ch4,Waste - Solid Waste:CH4,Waste - Solid Waste,Waste,emission_co2e_ch4_waso_biogas_food:emission_co...
55,waso,n2o,Waste - Solid Waste:N2O,Waste - Solid Waste,Waste,emission_co2e_n2o_waso_compost_food:emission_c...
56,trww,ch4,Waste - Wastewater Treatment:CH4,Waste - Wastewater Treatment,Waste,emission_co2e_ch4_trww_treated_advanced_aerobi...
57,trww,n2o,Waste - Wastewater Treatment:N2O,Waste - Wastewater Treatment,Waste,emission_co2e_n2o_trww_treated_advanced_aerobi...


In [ ]:
# ssp_edgar_df[ssp_edgar_df.ignore == 1]

AttributeError: 'DataFrame' object has no attribute 'ignore'

In [14]:
ssp_emissions_report, model_failed_flag = dru.generate_ssp_emissions_report(ssp_out)
ssp_emissions_report

,subsector,gas,edgar_class,ssp_emission
0,lvst,ch4,AG - Livestock:CH4,1.938988
1,lsmm,ch4,AG - Livestock:CH4,0.040259
2,lsmm,n2o,AG - Livestock:N2O,0.165996
3,agrc,co2,AG - Crops:CO2,1.316953
4,agrc,ch4,AG - Crops:CH4,0.214505
...,...,...,...,...
63,soil,co2,LULUCF - Organic Soil:CO2,0.110122
64,soil,n2o,LULUCF - Organic Soil:N2O,1.031985
65,ccsq,ch4,CCSQ:CH4,0.000000
66,ccsq,co2,CCSQ:CO2,0.000000


In [15]:
ssp_emissions_report = dru.group_ssp_emissions_report_vars(ssp_emissions_report)
ssp_emissions_report

,subsector,edgar_class,ssp_emission
0,agrc,AG - Crops:CH4,0.214505
1,agrc,AG - Crops:CO2,1.316953
2,agrc,AG - Crops:N2O,0.247102
3,ccsq,CCSQ:CH4,0.000000
4,ccsq,CCSQ:CO2,0.000000
5,ccsq,CCSQ:N2O,0.000000
6,entc,EN - Electricity/Heat:CH4,0.000000
7,entc,EN - Electricity/Heat:CO2,0.000000
8,entc,EN - Electricity/Heat:N2O,0.000000
9,fgtv,EN - Fugitive Emissions:CH4,0.000000


In [16]:
# check energy subsector
energy_subsectors = ['trns', 'entc', 'fgtv', 'scoe', 'inen', 'ccsq']
ssp_emissions_report[ssp_emissions_report.subsector.isin(energy_subsectors)]

,subsector,edgar_class,ssp_emission
3,ccsq,CCSQ:CH4,0.000000
4,ccsq,CCSQ:CO2,0.000000
5,ccsq,CCSQ:N2O,0.000000
6,entc,EN - Electricity/Heat:CH4,0.000000
7,entc,EN - Electricity/Heat:CO2,0.000000
8,entc,EN - Electricity/Heat:N2O,0.000000
9,fgtv,EN - Fugitive Emissions:CH4,0.000000
10,fgtv,EN - Fugitive Emissions:CO2,0.000000
11,fgtv,EN - Fugitive Emissions:N2O,0.000000
14,inen,EN - Manufacturing/Construction:CH4,0.010258


In [19]:
model_failed_flag

False

In [20]:
edgar_emissions.sort_values(by=['edgar_class'], ascending=True)

,iso_alpha_3,edgar_class,edgar_emission,year
2,HRV,AG - Crops:CH4,0.000227,2015
3,HRV,AG - Crops:CO2,0.095857,2015
4,HRV,AG - Crops:N2O,1.038055,2015
0,HRV,AG - Livestock:CH4,1.682916,2015
1,HRV,AG - Livestock:N2O,0.110193,2015
5,HRV,EN - Building:CH4,0.416708,2015
6,HRV,EN - Building:CO2,2.726867,2015
7,HRV,EN - Building:N2O,0.104122,2015
8,HRV,EN - Electricity/Heat:CH4,0.004396,2015
9,HRV,EN - Electricity/Heat:CO2,3.297328,2015


In [21]:
df_report = dru.merge_ssp_with_edgar(ssp_emissions_report, edgar_emissions)
df_report


,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year
0,agrc,AG - Crops:CH4,0.214505,HRV,0.000227,2015
1,agrc,AG - Crops:CO2,1.316953,HRV,0.095857,2015
2,agrc,AG - Crops:N2O,0.247102,HRV,1.038055,2015
3,ccsq,CCSQ:CH4,0.000000,NaN,NaN,2015
4,ccsq,CCSQ:CO2,0.000000,NaN,NaN,2015
5,ccsq,CCSQ:N2O,0.000000,NaN,NaN,2015
6,entc,EN - Electricity/Heat:CH4,0.000000,HRV,0.004396,2015
7,entc,EN - Electricity/Heat:CO2,0.000000,HRV,3.297328,2015
8,entc,EN - Electricity/Heat:N2O,0.000000,HRV,0.011222,2015
9,fgtv,EN - Fugitive Emissions:CH4,0.000000,HRV,1.046727,2015


In [117]:
dru.group_trns_scoe_inen(df_report)

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year
0,inen,EN - Manufacturing/Construction,4.069753,HRV,2.157340,2015
1,scoe,EN - Building,7.547604,HRV,3.247697,2015
2,trns,EN - Transportation,3.826507,HRV,5.958179,2015
3,agrc,AG - Crops:CH4,0.214505,HRV,0.000227,2015
4,agrc,AG - Crops:CO2,0.100414,HRV,0.095857,2015
5,agrc,AG - Crops:N2O,0.247102,HRV,1.038055,2015
6,ccsq,CCSQ:CH4,0.000000,NaN,NaN,2015
7,ccsq,CCSQ:CO2,0.000000,NaN,NaN,2015
8,ccsq,CCSQ:N2O,0.000000,NaN,NaN,2015
9,entc,EN - Electricity/Heat:CH4,0.000000,HRV,0.004396,2015


In [22]:
# get duplicated edgar_class
df_report[df_report.duplicated(subset=['edgar_class'], keep=False)]

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year


In [24]:
reports_dict = dru.run_report_generator(edgar_emissions, ssp_out)

sectoral_emission_report = reports_dict['sectoral_emission_report']
subsector_emission_report = reports_dict['subsector_emission_report']

In [25]:
sectoral_emission_report[sectoral_emission_report['subsector'].isin(energy_subsectors)]

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,4.069753,HRV,2.157340,2015,2.157341,0.886467,3.657318,2.157341,0.078574,1.149730
1,scoe,EN - Building,7.547604,HRV,3.247697,2015,3.247698,1.323986,18.489194,3.247698,0.118287,1.446377
2,trns,EN - Transportation,3.826507,HRV,5.958179,2015,5.958180,-0.357772,4.544028,5.958180,0.217008,1.939918


In [26]:
sectoral_emission_report

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,4.069753,HRV,2.157340,2015,2.157341,0.886467,3.657318e+00,2.157341,0.078574,1.149730
1,scoe,EN - Building,7.547604,HRV,3.247697,2015,3.247698,1.323986,1.848919e+01,3.247698,0.118287,1.446377
2,trns,EN - Transportation,3.826507,HRV,5.958179,2015,5.958180,-0.357772,4.544028e+00,5.958180,0.217008,1.939918
3,agrc,AG - Crops:CH4,0.214505,HRV,0.000227,2015,0.000228,941.296070,4.591488e-02,0.000228,0.000008,0.000227
4,agrc,AG - Crops:CO2,1.316953,HRV,0.095857,2015,0.095858,12.738568,1.491074e+00,0.095858,0.003491,0.091537
5,agrc,AG - Crops:N2O,0.247102,HRV,1.038055,2015,1.038056,-0.761957,6.256084e-01,1.038056,0.037808,0.711996
16,frst,LULUCF - Forest Land:CO2,-1.247014,HRV,-6.547104,2015,-6.547103,-0.809532,2.809094e+01,6.547105,0.238458,2.021164
17,ippu,IN - Industrial Processes:CH4,0.034023,HRV,0.001673,2015,0.001674,19.323149,1.046427e-03,0.001674,0.000061,0.001672
18,ippu,IN - Industrial Processes:CO2,7.361243,HRV,2.415891,2015,2.415892,2.047009,2.445650e+01,2.415892,0.087991,1.228438
19,ippu,IN - Industrial Processes:HFC,1.219243,HRV,0.514149,2015,0.514150,1.371375,4.971556e-01,0.514150,0.018726,0.414854


In [27]:
sectoral_emission_report.tail(10)

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
24,lndu,LULUCF - Deforestation:CO2,0.242741,HRV,0.060840,2015,0.060841,2.989786,0.033088,0.060841,0.002216,0.059061
25,lndu,LULUCF - Other Land:CO2,0.025623,HRV,0.820237,2015,0.820238,-0.968761,0.631413,0.820238,0.029875,0.598967
26,lsmm,AG - Livestock:N2O,0.165996,HRV,0.110193,2015,0.110194,0.506401,0.003114,0.110194,0.004013,0.104534
27,lvst-lsmm,AG - Livestock:CH4,1.979247,HRV,1.682916,2015,1.682917,0.176081,0.087812,1.682917,0.061295,0.986904
28,soil,LULUCF - Organic Soil:CO2,0.110122,HRV,0.092259,2015,0.092260,0.193601,0.000319,0.092260,0.003360,0.088248
30,trww,Waste - Wastewater Treatment:CH4,0.394961,HRV,0.441746,2015,0.441747,-0.105912,0.002189,0.441747,0.016089,0.365855
31,trww,Waste - Wastewater Treatment:N2O,0.270326,HRV,0.061427,2015,0.061428,3.400672,0.043638,0.061428,0.002237,0.059614
32,waso,Waste - Solid Waste:CH4,0.832718,HRV,1.255624,2015,1.255625,-0.336810,0.178851,1.255625,0.045732,0.813427
33,waso,Waste - Solid Waste:CO2,0.467341,HRV,0.000031,2015,0.000032,14667.292538,0.218377,0.000032,0.000001,0.000031
34,waso,Waste - Solid Waste:N2O,0.000689,HRV,0.003034,2015,0.003035,-0.772939,0.000006,0.003035,0.000111,0.003030


In [28]:
sectoral_emission_report[sectoral_emission_report['subsector'] == 'ippu']

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
17,ippu,IN - Industrial Processes:CH4,0.034023,HRV,0.001673,2015,0.001674,19.323149,1.046427e-03,0.001674,0.000061,0.001672
18,ippu,IN - Industrial Processes:CO2,7.361243,HRV,2.415891,2015,2.415892,2.047009,2.445650e+01,2.415892,0.087991,1.228438
19,ippu,IN - Industrial Processes:HFC,1.219243,HRV,0.514149,2015,0.514150,1.371375,4.971556e-01,0.514150,0.018726,0.414854
20,ippu,IN - Industrial Processes:N2O,2.553071,HRV,0.946129,2015,0.946130,1.698437,2.582262e+00,0.946130,0.034460,0.665842
21,ippu,IN - Industrial Processes:PFC,0.000008,HRV,0.000000,2015,0.000001,7.248265,5.253735e-11,0.000001,0.000000,0.000000
22,ippu,IN - Industrial Processes:SF6,0.000000,HRV,0.005376,2015,0.005377,-1.000000,2.891163e-05,0.005377,0.000196,0.005362


In [29]:
subsector_emission_report

,subsector,ssp_emission,edgar_emission_epsilon,rel_error,squared_diff,year
0,agrc,1.778561,1.134142,0.568199,0.415276,2015
1,frst,-1.247014,-6.547103,-0.809532,28.090943,2015
2,inen,4.069753,2.157341,0.886467,3.657318,2015
3,ippu,11.167588,3.883223,1.875855,53.061963,2015
4,lndu,0.268364,0.881078,-0.695415,0.375419,2015
5,lsmm,0.165996,0.110194,0.506401,0.003114,2015
6,lvst-lsmm,1.979247,1.682917,0.176081,0.087812,2015
7,scoe,7.547604,3.247698,1.323986,18.489194,2015
8,soil,0.110122,0.092260,0.193601,0.000319,2015
9,trns,3.826507,5.958180,-0.357772,4.544028,2015


### Test new error function

In [125]:
input_df = pd.read_csv('misc/dummy/ssp_output_dummy.csv')
input_df.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,128992.475401,286925.570284,59534.988647,155176.845408,200241.524314,109285.291660,144565.284932,96468.731604,66425.612333,...,860714.235864,5.781814e+06,65829.816853,279108.581486,50211.974800,175242.943363,0.0,0.0,2.050396e+06,725849.288265
1,1,128305.935425,285398.458926,59218.124042,154350.943684,199175.773688,108703.639735,143795.861390,95955.293587,66072.073584,...,857478.660027,5.906717e+06,52166.876709,331312.854435,70450.376552,215192.338859,2736.0,2449.0,2.713049e+06,699533.070417
2,2,127621.990578,283877.120067,58902.457190,153528.163879,198114.051612,108124.186462,143029.346278,95443.796372,65719.871216,...,660924.245615,5.951148e+06,38967.956801,296921.771405,82224.830575,210243.344677,2736.0,2449.0,2.331104e+06,761148.281366
3,3,126940.639041,282361.549661,58587.987250,152708.503803,197056.355263,107546.930298,142265.737557,94934.238599,65369.004292,...,957822.768981,5.919376e+06,17239.772927,378730.362142,93529.211867,213539.618942,2736.0,2449.0,2.111415e+06,729155.981592
4,4,126261.879236,280851.744198,58274.713494,151891.961560,196002.682190,106971.869908,141505.033460,94426.619087,65019.472000,...,892950.504956,5.887725e+06,21001.252360,359927.486275,85408.876964,264501.812874,2736.0,2449.0,2.234268e+06,737682.556359


In [126]:
dru.run_report_generator(edgar_emissions, input_df)

Index(['subsector', 'gas', 'edgar_class', 'edgar_subsector', 'edgar_sector',
       'ignore', 'note', 'need_better_information_on_what_is_contained',
       'vars'],
      dtype='object')
Missing variables for lvst False
Missing variables for lsmm False
Missing variables for lsmm False
Missing variables for agrc False
Missing variables for agrc False
Missing variables for agrc False
Missing variables for scoe False
Missing variables for scoe False
Missing variables for scoe False
Missing variables for entc True
Missing variables for entc True
Missing variables for entc True
Missing variables for fgtv True
Missing variables for fgtv True
Missing variables for fgtv True
Missing variables for inen False
Missing variables for inen False
Missing variables for inen False
Missing variables for trns False
Missing variables for trns False
Missing variables for trns False
Missing variables for ippu False
Missing variables for ippu False
Missing variables for ippu False
Missing variables for ippu

In [127]:
df = dru.sectoral_emission_report.copy()
df.head()

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
0,inen,EN - Manufacturing/Construction,2.808345,HRV,2.157340,2015,2.157341,0.301762,0.423806,2.157341,0.078574,1.149730
1,scoe,EN - Building,4.064501,HRV,3.247697,2015,3.247698,0.251502,0.667166,3.247698,0.118287,1.446377
2,trns,EN - Transportation,6.009059,HRV,5.958179,2015,5.958180,0.008539,0.002589,5.958180,0.217008,1.939918
3,agrc,AG - Crops:CH4,0.290094,HRV,0.000227,2015,0.000228,1273.348921,0.084023,0.000228,0.000008,0.000227
4,agrc,AG - Crops:CO2,0.065572,HRV,0.095857,2015,0.095858,-0.315950,0.000917,0.095858,0.003491,0.091537


In [128]:
df.to_csv('misc/dummy/sectoral_emission_report_dummy.csv', index=False)

In [129]:
df.sort_values(by=['norm_weight'], ascending=False, inplace=True)

In [130]:
df

,subsector,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,rel_error,squared_diff,direct_weight,norm_weight,log_weight
16,frst,LULUCF - Forest Land:CO2,-1.650303,HRV,-6.547104,2015,-6.547103,-0.747934,2.397865e+01,6.547105,0.238458,2.021164
2,trns,EN - Transportation,6.009059,HRV,5.958179,2015,5.958180,0.008539,2.588703e-03,5.958180,0.217008,1.939918
1,scoe,EN - Building,4.064501,HRV,3.247697,2015,3.247698,0.251502,6.671663e-01,3.247698,0.118287,1.446377
18,ippu,IN - Industrial Processes:CO2,3.669455,HRV,2.415891,2015,2.415892,0.518883,1.571422e+00,2.415892,0.087991,1.228438
0,inen,EN - Manufacturing/Construction,2.808345,HRV,2.157340,2015,2.157341,0.301762,4.238060e-01,2.157341,0.078574,1.149730
27,lvst-lsmm,AG - Livestock:CH4,1.658163,HRV,1.682916,2015,1.682917,-0.014709,6.127554e-04,1.682917,0.061295,0.986904
32,waso,Waste - Solid Waste:CH4,0.829621,HRV,1.255624,2015,1.255625,-0.339276,1.814796e-01,1.255625,0.045732,0.813427
5,agrc,AG - Crops:N2O,0.377462,HRV,1.038055,2015,1.038056,-0.636377,4.363853e-01,1.038056,0.037808,0.711996
20,ippu,IN - Industrial Processes:N2O,1.276536,HRV,0.946129,2015,0.946130,0.349219,1.091682e-01,0.946130,0.034460,0.665842
25,lndu,LULUCF - Other Land:CO2,0.014875,HRV,0.820237,2015,0.820238,-0.981865,6.486093e-01,0.820238,0.029875,0.598967


In [131]:
df['norm_weight'] * 100

16    23.845818
2     21.700839
1     11.828741
18     8.799141
0      7.857450
27     6.129504
32     4.573227
5      3.780798
20     3.445983
25     2.987461
19     1.872631
30     1.608924
26     0.401343
4      0.349130
28     0.336027
31     0.223730
24     0.221589
22     0.019580
34     0.011051
17     0.006094
3      0.000825
33     0.000112
21     0.000000
Name: norm_weight, dtype: float64

In [132]:
ef = ErrorFunctions()
ef.calculate_error(error_type='wmape', dataframe=df)

1.9768234816325139

In [133]:
np.sum(df['norm_weight'] * df['rel_error'].abs()) * 100

45.46694007754782